In [1]:
import pandas as pd
import numpy as np
import os
import sys
import Mixture
import multiprocessing
from joblib import Parallel, delayed
from random import seed
from random import sample 
from random import randint

In [3]:
lines = [2, 6]
linestil7 = [2, 4]
lineslm22 = [2, 11]
cpu = multiprocessing.cpu_count()
rango = 1000

In [ ]:
dataFrameSignature = pd.read_excel('data/TIL9_signature.xlsx', sheet_name = 0)

In [18]:
dataFrameSignature.iloc[:, 1:]

,CD4_Tcell,NK_cell,M2,M1,DC_cell,T_helper,Treg,CD8_Tcell,memory_CD8_Tcell
0,16.559945,15.033687,18.237954,19.779773,16.850401,14.328851,2329.722651,14.830404,351.856901
1,10.755308,14.285563,12.803071,12.061551,14.134501,11.571460,969.054167,10.679765,405.502781
2,67.241080,45.188069,154.226407,176.827396,74.102202,66.564266,53.089329,59.296827,27.840216
3,64.026800,140.379261,7737.629850,5052.409992,4855.427133,238.532329,35.825204,153.109289,53.741866
4,89.887276,70.625413,9184.516116,3922.877311,5024.291661,78.190748,20.465571,45.001900,39.423679
...,...,...,...,...,...,...,...,...,...
12360,14.441113,12.519361,14.597253,15.651890,12.400772,10.653762,21.418106,11.000796,20.075507
12361,8.866926,26.743882,7.462365,8.039629,8.341252,8.351226,19.574601,7.914022,10.697394
12362,9.162524,10.077110,13.583478,8.900550,19.176400,11.413584,7.494052,10.566209,7.454016
12363,13.436374,27.822516,26.511180,29.204494,17.208578,22.350946,86.675119,15.398554,71.872377


In [17]:
dataFrameSignature

,Gene symbol,CD4_Tcell,NK_cell,M2,M1,DC_cell,T_helper,Treg,CD8_Tcell,memory_CD8_Tcell
0,ARID5B,16.559945,15.033687,18.237954,19.779773,16.850401,14.328851,2329.722651,14.830404,351.856901
1,ZNF460,10.755308,14.285563,12.803071,12.061551,14.134501,11.571460,969.054167,10.679765,405.502781
2,RBM47,67.241080,45.188069,154.226407,176.827396,74.102202,66.564266,53.089329,59.296827,27.840216
3,GRN,64.026800,140.379261,7737.629850,5052.409992,4855.427133,238.532329,35.825204,153.109289,53.741866
4,CTSB,89.887276,70.625413,9184.516116,3922.877311,5024.291661,78.190748,20.465571,45.001900,39.423679
...,...,...,...,...,...,...,...,...,...,...
12360,OR2L2,14.441113,12.519361,14.597253,15.651890,12.400772,10.653762,21.418106,11.000796,20.075507
12361,GCSAML,8.866926,26.743882,7.462365,8.039629,8.341252,8.351226,19.574601,7.914022,10.697394
12362,TMEM231,9.162524,10.077110,13.583478,8.900550,19.176400,11.413584,7.494052,10.566209,7.454016
12363,ZNF155,13.436374,27.822516,26.511180,29.204494,17.208578,22.350946,86.675119,15.398554,71.872377


In [21]:
def createBetas (X, a, b, n):

    ns = randint(a, b)
    r = np.random.uniform(1, 0.2, ns)
    rn = r/r.sum()
    id = sample(range(n-1), ns)
    betas = np.repeat(.0, n, axis=0)
    for index, i in enumerate(id, start = 0):
        betas[i] = rn[index]

    vector = X.to_numpy(copy=True)    
    vector = vector.flatten()
        
    A = np.dot(X, betas)# + vector[sample(range(len(vector)), len(X.index))]
    print(np.dot(X, betas).shape)

    return betas, id, A

In [ ]:
X = dataFrameSignature.iloc[:, 1:]

subjects = Parallel(n_jobs=cpu, backend='threading')(delayed(createBetas)(X = X, a = lines[0], b = lines[1], n = len(X.columns)) for i in range(rango))

# Getting expression matrix
vector = [x[2] for x in subjects]
columns = ['V' + str(x+1) for x in range(len(subjects))]
Y2 = pd.DataFrame(np.column_stack(vector), columns=columns)
Y2.insert(0, 'Gene symbol', dataFrameSignature['Gene symbol'])

# Getting Real Betas Matrix
vector = [x[0] for x in subjects]
columns = ['V' + str(x+1) for x in range(len(subjects))]
betas = pd.DataFrame(np.column_stack(vector), columns=columns)

X = dataFrameSignature

vector = [len(x[1]) for x in subjects]
ids = pd.DataFrame(np.column_stack(vector))   

In [28]:
len(subjects)

1000

In [15]:
betas

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,...,V991,V992,V993,V994,V995,V996,V997,V998,V999,V1000
0,0.192692,0.000000,0.000000,0.117930,0.000000,0.000000,0.506784,0.191232,0.187124,0.284775,...,0.000000,0.191460,0.000000,0.216760,0.000000,0.41642,0.175044,0.428474,0.560421,0.228989
1,0.138896,0.000000,0.000000,0.145800,0.000000,0.000000,0.000000,0.000000,0.264848,0.000000,...,0.000000,0.065641,0.157240,0.000000,0.251181,0.00000,0.273681,0.000000,0.000000,0.000000
2,0.164369,0.073941,0.712403,0.170045,0.594451,0.441608,0.163731,0.088504,0.220902,0.000000,...,0.000000,0.162186,0.239686,0.187356,0.000000,0.00000,0.000000,0.000000,0.439579,0.173168
3,0.000000,0.173630,0.000000,0.089689,0.249587,0.000000,0.000000,0.212518,0.116940,0.186555,...,0.000000,0.000000,0.284172,0.000000,0.198222,0.00000,0.153404,0.000000,0.000000,0.109832
4,0.095986,0.254480,0.000000,0.000000,0.000000,0.000000,0.329484,0.146979,0.131209,0.092146,...,0.376298,0.286497,0.318902,0.183977,0.000000,0.00000,0.205184,0.000000,0.000000,0.000000
5,0.000000,0.201994,0.287597,0.241608,0.000000,0.558392,0.000000,0.211889,0.078976,0.000000,...,0.293279,0.000000,0.000000,0.000000,0.000000,0.00000,0.192687,0.000000,0.000000,0.272262
6,0.282611,0.295955,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.160575,...,0.330423,0.214741,0.000000,0.289428,0.262470,0.00000,0.000000,0.000000,0.000000,0.215749
7,0.125446,0.000000,0.000000,0.234927,0.155962,0.000000,0.000000,0.148879,0.000000,0.275949,...,0.000000,0.079476,0.000000,0.122480,0.288127,0.58358,0.000000,0.571526,0.000000,0.000000
8,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000


In [16]:
X

,Gene symbol,CD4_Tcell,NK_cell,M2,M1,DC_cell,T_helper,Treg,CD8_Tcell,memory_CD8_Tcell
0,ARID5B,16.559945,15.033687,18.237954,19.779773,16.850401,14.328851,2329.722651,14.830404,351.856901
1,ZNF460,10.755308,14.285563,12.803071,12.061551,14.134501,11.571460,969.054167,10.679765,405.502781
2,RBM47,67.241080,45.188069,154.226407,176.827396,74.102202,66.564266,53.089329,59.296827,27.840216
3,GRN,64.026800,140.379261,7737.629850,5052.409992,4855.427133,238.532329,35.825204,153.109289,53.741866
4,CTSB,89.887276,70.625413,9184.516116,3922.877311,5024.291661,78.190748,20.465571,45.001900,39.423679
...,...,...,...,...,...,...,...,...,...,...
12360,OR2L2,14.441113,12.519361,14.597253,15.651890,12.400772,10.653762,21.418106,11.000796,20.075507
12361,GCSAML,8.866926,26.743882,7.462365,8.039629,8.341252,8.351226,19.574601,7.914022,10.697394
12362,TMEM231,9.162524,10.077110,13.583478,8.900550,19.176400,11.413584,7.494052,10.566209,7.454016
12363,ZNF155,13.436374,27.822516,26.511180,29.204494,17.208578,22.350946,86.675119,15.398554,71.872377
